In [ ]:
import pandas as pd
from functools import reduce

# Baca file Excel
data = pd.read_csv('/content/911_Calls_for_Service_(Last_30_Days).csv')

# Filter baris dengan data yang hilang di kolom Zip atau Neighborhood
filtered_data = data.dropna(subset=['zip_code', 'neighborhood'])

# Konversi ke dalam bentuk daftar dictionary
filtered_dict = filtered_data.to_dict(orient='records')

# Filter data untuk kepolisian Detroit
detroit_data = filter(lambda x: 'Detroit' in x['neighborhood'], filtered_dict)

# Fungsi untuk menghitung total waktu respons, waktu pengiriman, dan total waktu rata-rata
def calculate_averages(acc, curr):
    acc['total_response_time'] += curr['totalresponsetime']
    acc['total_dispatch_time'] += curr['dispatchtime']
    acc['total_time_on_scene'] += curr['time_on_scene']
    acc['count'] += 1
    return acc

# Inisialisasi nilai awal
initial_values = {'total_response_time': 0, 'total_dispatch_time': 0, 'total_time_on_scene': 0, 'count': 0}

# Hitung total waktu respons, waktu pengiriman, dan total waktu rata-rata
totals = reduce(calculate_averages, detroit_data, initial_values)

# Hitung rata-rata
average_response_time = totals['total_response_time'] / totals['count']
average_dispatch_time = totals['total_dispatch_time'] / totals['count']
average_time_on_scene = totals['total_time_on_scene'] / totals['count']


In [ ]:
from functools import reduce

# Fungsi untuk menghitung total waktu respons, waktu pengiriman, dan total waktu rata-rata untuk setiap neighborhood
def calculate_neighborhood_averages(acc, curr):
    neighborhood = curr[0]['neighborhood']
    total_response_time = sum(map(lambda x: x['totalresponsetime'], curr))
    total_dispatch_time = sum(map(lambda x: x['dispatchtime'], curr))
    total_time_on_scene = sum(map(lambda x: x['time_on_scene'], curr))
    count = len(curr)

    acc.append({
        'neighborhood': neighborhood,
        'total_response_time': total_response_time,
        'total_dispatch_time': total_dispatch_time,
        'total_time_on_scene': total_time_on_scene,
        'count': count,
        'average_response_time': total_response_time / count,
        'average_dispatch_time': total_dispatch_time / count,
        'average_time_on_scene': total_time_on_scene / count
    })
    return acc

# Bagi list dictionary menjadi list dictionary yang lebih kecil yang dipisahkan oleh neighborhood
neighborhood_data = map(lambda neighborhood: list(filter(lambda x: x['neighborhood'] == neighborhood, filtered_dict)), set(map(lambda x: x['neighborhood'], filtered_dict)))

# Hitung total waktu respons, waktu pengiriman, dan total waktu rata-rata untuk setiap neighborhood
neighborhood_averages = reduce(calculate_neighborhood_averages, neighborhood_data, [])

# Tambahkan item dictionary untuk menyertakan data populasi untuk semua Detroit
total_detroit_population = {
    'neighborhood': 'All Detroit',
    'total_response_time': totals['total_response_time'],
    'total_dispatch_time': totals['total_dispatch_time'],
    'total_time_on_scene': totals['total_time_on_scene'],
    'count': totals['count'],
    'average_response_time': average_response_time,
    'average_dispatch_time': average_dispatch_time,
    'average_time_on_scene': average_time_on_scene
}
neighborhood_averages.append(total_detroit_population)


In [ ]:
import json

# Tulis data ke dalam file JSON
with open('output.json', 'w') as f:
    json.dump(neighborhood_averages, f, indent=4)
